<a href="https://colab.research.google.com/github/Voodoo-999/gpt_from_scratch/blob/main/gpt_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F



In [ ]:
# hyper param
batch_size = 64
block_size = 256
n_embd = 384
eval_interval = 500
max_iters = 5000
learning_rate = 3e-4
n_head = 6
n_layer = 6
dropout =0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:

filename = list(uploaded.keys())[0]
with open(filename,'r') as f:
  text = f.read()

text_len=len(text)
print(text_len)
vocab = sorted(list(set(text)))
print(vocab)
vocab_len = len(vocab)
print(vocab_len)

In [ ]:
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for i, ch in enumerate(vocab)}
def encode(string):
  return [stoi[c] for c in string]
def decode(integers):
  return ''.join(itos[i] for i in integers)

In [ ]:
encoded_text = torch.tensor(encode(text), dtype = torch.long)

print(encoded_text.shape)

In [ ]:
train_split = int(0.9*text_len)
train_text = encoded_text[:train_split]
val_text = encoded_text[train_split:]

In [ ]:
def get_batch(split):
  if split == 'train':
    data = train_text
  else:
    data = val_text
  idx = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx])
  x, y = x.to(device), y.to(device)
  return x, y

xb, yb = get_batch('train')
print(xb)
print(yb)

In [ ]:
class Head(nn.Module):
  #one head of self attention

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size,bias=False)
    self.value = nn.Linear(n_embd,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x) # (B,T,C)
    q = self.query(x)# (B,T,C)
    v = self.value(x)# (B,T,C)
    wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,C) * (B,C,T) --> (B,T,T)
    wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # still (B,T,T)
    wei = F.softmax(wei, dim =-1)
    wei = self.dropout(wei)
    out = wei @ v # (B,T,T) * (B,T,C) --> (B,T,C)
    return out


In [ ]:
class MultiHeadattention(nn.Module):
  # multi headed attention

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)


  def forward(self, x):
    out =  torch.cat([h(x) for h in self.heads], dim =-1)
    out = self.dropout(self.proj(out))
    return out

In [ ]:
class FeedFoward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(), # Add parentheses here to instantiate the module
        # this is the projection going back to the residual pathway
        nn.Linear(4*n_embd, n_embd),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

In [ ]:
class Block(nn.Module):
  def __init__(self,n_embd, n_head):
    super().__init__()
    head_size = n_embd//n_head
    self.sa_heads = MultiHeadattention(n_head,head_size)
    self.ffwd = FeedFoward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa_heads(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [ ]:
class GPTLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embd_table = nn.Embedding(vocab_len, n_embd)
    self.position_embd_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_len)

  def forward(self, idx, targets = None):
    B, T = idx.shape

    tok_emb = self.token_embd_table(idx) # (B, T, n_emd)
    pos_emb = self.position_embd_table(torch.arange(T, device=device)) #(T,C)
    x = tok_emb + pos_emb
    x = self.blocks(x)
    logits = self.lm_head(x) # (B, T, vocab_size)
    if targets is None :
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss
  def generate(self, idx, max_new_tokens):
    for _ in range (max_new_tokens):
      idx_cropped = idx[:, -block_size:]
      logits, _loss = self(idx_cropped)
      logits = logits[:,-1,:] #becomes (B,C)
      probs = F.softmax(logits, dim=-1) # (B,C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
      idx = torch.cat((idx, idx_next), dim= 1) # (B,T+1)
    return idx
model = GPTLanguageModel()
m= model.to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

In [ ]:
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long, device = device), max_new_tokens=1000)[0].tolist()))


In [ ]:
@torch.no_grad()
def estimate_loss(gpt, iters = 100):
  out = {}
  gpt.eval()
  for split in ['train','val']:
    losses = torch.zeros(iters)
    for k in range(iters):
      X, Y = get_batch(split)
      _logits, loss = gpt(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  gpt.train()
  return out

In [ ]:

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

def train_model(model, optimizer):
  for iter in range(max_iters):
    if iter % eval_interval == 0:
      loss_status = estimate_loss(model)
      print(f"train loss: {loss_status['train'].item():.4f}, eval loss: {loss_status['val'].item():.4f} ")
    x, y = get_batch('train')
    logits, loss = m(x,y)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
train_model(m,optimizer)

In [ ]:
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long, device = device), max_new_tokens=1000)[0].tolist()))
